In [1]:
import pandas as pd
import networkx as nx

In [4]:
# Get nodes for bipartite network
users = pd.read_csv("nodelist2_poli.csv")
# tweets = pd.read_csv("tweet_nodes.csv")
users.head()

<ipython-input-4-3f3f8eb7392e>:2: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv("nodelist2_poli.csv")


,userid,username,textid,location,language,tweetid,tweetcreatedts,retweetcount,hashtags,text,...,totaltweets,tags_splitted,tags_lower,pro_Russian,pro_Ukraine,Neutral,NodeType,nodeInG1,nodeInG2,ID
0,4832941924,Mari_Berbec,1.0,"Berlin, Germany",en,1497476520742309891,2022-02-26 07:39:58,0,"[{'text': 'putin', 'indices': [53, 59]}, {'tex...",@charlieweissr Socialism has nothing to do wit...,...,2848.0,"['putin', 'trump']","['putin', 'trump']",False,False,True,Neutral,True,True,4.832942e+09
1,412597196,plaroch,7.0,NaN,fr,1496862205584855054,2022-02-24 14:58:54,0,"[{'text': 'Ukraine', 'indices': [32, 40]}, {'t...",Ce fut une terrible erreur de l'#Ukraine de se...,...,141300.0,"['Ukraine', 'Poutine']","['ukraine', 'poutine']",False,False,True,Neutral,True,True,4.125972e+08
2,1494370657152675840,touchsoundart,11.0,Macau,en,1497725055920345088,2022-02-27 00:07:34,1,"[{'text': 'nft', 'indices': [76, 80]}, {'text'...","Hello friends, I invite you for a walk around ...",...,53.0,"['nft', '35mm', 'NFTCommunity', 'Ukraine']","['nft', '35mm', 'nftcommunity', 'ukraine']",False,False,True,Neutral,True,True,1.494371e+18
3,2573523092,deliciousndprvd,17.0,NaN,en,1496849491424694279,2022-02-24 14:08:23,2208,"[{'text': 'Ukraine', 'indices': [37, 45]}, {'t...",To all of those in #Ukraine who are looking fo...,...,38050.0,"['Ukraine', 'Poland']","['ukraine', 'poland']",False,False,True,Neutral,True,True,2.573523e+09
4,1489178054513545216,poposogolo,22.0,NaN,it,1497219893866999809,2022-02-25 14:40:14,15,"[{'text': 'Difesa', 'indices': [70, 77]}]",Con l’incarico di Segretario del Consiglio Sup...,...,2261.0,['Difesa'],['difesa'],False,False,False,Unknown,True,True,1.489178e+18


In [4]:
print(users.head())
print(tweets.head())

   Unnamed: 0               userid         username   location  \
0       58294  1273101694268837890     nowhere_safe    Web3.0    
1       58303   724605084169060353  VillTjanaPengar        NaN   
2       58311             19535996       stemdesign     London   
3       58339   861486301211709440  quentinhubert99   Tergnier   
4       58340             51813322          EduA001  Venezuela   

                usercreatedts  following  followers  totaltweets  
0  2020-06-17 03:54:42.000000       2421        798         1693  
1  2016-04-25 14:24:49.000000        119          1            1  
2  2009-01-26 14:22:06.000000        591        363         4284  
3  2017-05-08 07:41:54.000000        131        136         6102  
4  2009-06-28 19:20:19.000000       1371       2949       102887  
   Unnamed: 0  textid                                               text  \
0       58294   58294  I learn to use #Kalashnikov and prepare to bea...   
1       58303   58303  .@ZelenskyyUa's tv address

In [5]:
# drop the first column (old indices)
users = users.drop(['Unnamed: 0'], axis=1)
tweets = tweets.drop(['Unnamed: 0'], axis=1)

In [6]:
# Get edge list for bipartite network
edges = pd.read_csv("edges.csv")

In [7]:
print(users.shape)
print(tweets.shape)
print(edges.shape)

(16315, 7)
(5333, 3)
(17255, 2)


In [9]:
print("User attributes:",users.columns.values)
print("Tweet attributes:",tweets.columns.values)

User attributes: ['userid' 'username' 'location' 'usercreatedts' 'following' 'followers'
 'totaltweets']
Tweet attributes: ['textid' 'text' 'hashtags']


In [10]:
# Create graph with nodes' attributes
B = nx.Graph()
B.add_nodes_from(users["userid"], bipartite=0, username=users["userid"],location=users["location"],
                 usercreatedts=users["usercreatedts"], following=users["following"],followers=users["followers"])
B.add_nodes_from(tweets["textid"], bipartite=1, text=tweets["text"], hashtags=tweets["hashtags"])
B.add_edges_from(edges.values)

In [11]:
from networkx.algorithms import bipartite

# Create one-mode networks
users_nodes = {n for n, a in B.nodes(data=True) if a["bipartite"] == 0}
tweets_nodes = set(B) - users_nodes

B_users = bipartite.projected_graph(B, users_nodes)
B_tweets = bipartite.projected_graph(B, tweets_nodes)

print(nx.info(B_users))
print(nx.info(B_tweets))

Graph with 16315 nodes and 256226 edges
Graph with 5244 nodes and 1114 edges


In [12]:
users_degcentrality = nx.degree_centrality(B_users)
tweets_degcentrality = nx.degree_centrality(B_tweets) 

In [14]:
# provide the summary information of two projected networks
print('Projected Network of Insects')
print(f'The network has {len(B_users.nodes)} nodes and {len(B_users.edges)} edges.')

print(f'The network is weighted:\t\t {nx.is_weighted(B_users)}')
print(f'The network is directed:\t\t {nx.is_directed(B_users)}')
print(f'The network is bipartite:\t\t {nx.is_bipartite(B_users)}')
print(f'The network is connected:\t\t {nx.is_connected(B_users)}')

print(f'The isolated node in the network:\t {len(list(nx.isolates(B_users)))}')
B_users_del = B_users.copy()
B_users_del.remove_nodes_from(list(nx.isolates(B_users_del))) # delete the isolated nodes

print('-------------')
print(f'After deleting the isolated nodes, the network has {len(B_users_del.nodes)} nodes and {len(B_users_del.edges)} edges.')
# create a subgraph that is the largest connected component
S_users_del = max(nx.connected_components(B_users_del), key=len)
S_users_del = B_users_del.subgraph(S_users_del)

B_users_del_comp = [len(c) for c in sorted(nx.connected_components(B_users_del), key=len, reverse=True)]
print(f'There are {len(B_users_del_comp)} connected components in the network and the largest one has {max(B_users_del_comp)} nodes.')

Projected Network of Insects
The network has 16315 nodes and 256226 edges.
The network is weighted:		 False
The network is directed:		 False
The network is bipartite:		 False
The network is connected:		 False
The isolated node in the network:	 2474
-------------
After deleting the isolated nodes, the network has 13841 nodes and 256226 edges.
There are 1920 connected components in the network and the largest one has 2585 nodes.


In [15]:
print('-------------')
print(f'Density:\t\t\t {nx.density(S_users_del)}')
print(f'Network diameter:\t\t {nx.diameter(S_users_del)}')
print(f'Average shortest path length:\t {nx.average_shortest_path_length(S_users_del)}')
print(f'Average degree:\t\t\t {sum([v for (u,v) in list(nx.degree(S_users_del))]) / len(S_users_del)}')
print('-------------')
degree_insect = dict(nx.degree_centrality(S_users_del))
print(f'Node with the highest degree centrality:\t\t {max(degree_insect, key=degree_insect.get)}')
print(f'Node with the lowest degree centrality:\t\t\t {min(degree_insect, key=degree_insect.get)}')
print('')
betweenness = dict(nx.betweenness_centrality(S_users_del))
print(f'Node with the highest betweenese degree centrality:\t {max(betweenness, key=betweenness.get)}')
print(f'Node with the lowest betweenese degree centrality:\t {min(betweenness, key=betweenness.get)}')
print('')
closeness = dict(nx.closeness_centrality(S_users_del))
print(f'Node with the highest closeness degree centrality:\t {max(closeness, key=closeness.get)}')
print(f'Node with the lowest closeness degree centrality:\t {min(closeness, key=closeness.get)}')
print('')
eigenvector = dict(nx.eigenvector_centrality(S_users_del))
print(f'Node with the highest eigenvector degree centrality:\t {max(eigenvector, key=eigenvector.get)}')
print(f'Node with the lowest eigenvector degree centrality:\t {min(eigenvector, key=eigenvector.get)}')
print('')
load = dict(nx.load_centrality(S_users_del))
print(f'Node with the highest load degree centrality:\t\t {max(load, key=load.get)}')
print(f'Node with the lowest load degree centrality:\t\t {min(load, key=load.get)}')

-------------
Density:			 0.041265397536394174
Network diameter:		 26
Average shortest path length:	 8.621868543813738
Average degree:			 106.62978723404255
-------------
Node with the highest degree centrality:		 417128105
Node with the lowest degree centrality:			 1405773541866233866

Node with the highest betweenese degree centrality:	 389950297
Node with the lowest betweenese degree centrality:	 942452818312683520

Node with the highest closeness degree centrality:	 3055684983
Node with the lowest closeness degree centrality:	 831140867369005056

Node with the highest eigenvector degree centrality:	 417128105
Node with the lowest eigenvector degree centrality:	 44092408

Node with the highest load degree centrality:		 389950297
Node with the lowest load degree centrality:		 942452818312683520


In [16]:
# provide the summary information of two projected networks
print('Projected Network of Tweets')
print(f'The network has {len(B_tweets.nodes)} nodes and {len(B_tweets.edges)} edges.')

print(f'The network is weighted:\t\t {nx.is_weighted(B_tweets)}')
print(f'The network is directed:\t\t {nx.is_directed(B_tweets)}')
print(f'The network is bipartite:\t\t {nx.is_bipartite(B_tweets)}')
print(f'The network is connected:\t\t {nx.is_connected(B_tweets)}')

print(f'The number of isolated nodes in the network:\t {len(list(nx.isolates(B_tweets)))}')
B_tweets_del = B_tweets.copy()
B_tweets_del.remove_nodes_from(list(nx.isolates(B_tweets_del))) # delete the isolated nodes

print('-------------')
print(f'After deleting the isolated nodes, the network has {len(B_tweets_del.nodes)} nodes and {len(B_tweets_del.edges)} edges.')

B_tweets_del_comp = [len(c) for c in sorted(nx.connected_components(B_tweets_del), key=len, reverse=True)]
print(f'There are {len(B_tweets_del_comp)} connected components in the network and the largest one has {max(B_tweets_del_comp)} nodes.')


Projected Network of Tweets
The network has 5244 nodes and 1114 edges.
The network is weighted:		 False
The network is directed:		 False
The network is bipartite:		 False
The network is connected:		 False
The number of isolated nodes in the network:	 4105
-------------
After deleting the isolated nodes, the network has 1139 nodes and 1114 edges.
There are 289 connected components in the network and the largest one has 288 nodes.


In [17]:
# create a subgraph that is the largest connected component
S_tweets_del = max(nx.connected_components(B_tweets_del), key=len)
S_tweets_del = B_tweets_del.subgraph(S_tweets_del)
print('-------------')
print(f'Density:\t\t\t {nx.density(S_tweets_del)}')
print(f'Network diameter:\t\t {nx.diameter(S_tweets_del)}')
print(f'Average shortest path length:\t {nx.average_shortest_path_length(S_tweets_del)}')
print(f'Average degree:\t\t\t {sum([v for (u,v) in list(nx.degree(S_tweets_del))]) / len(S_tweets_del)}')
print('-------------')
degree_insect = dict(nx.degree_centrality(S_tweets_del))
print(f'Node with the highest degree centrality:\t\t {max(degree_insect, key=degree_insect.get)}')
print(f'Node with the lowest degree centrality:\t\t\t {min(degree_insect, key=degree_insect.get)}')
print('')
betweenness = dict(nx.betweenness_centrality(S_tweets_del))
print(f'Node with the highest betweenese degree centrality:\t {max(betweenness, key=betweenness.get)}')
print(f'Node with the lowest betweenese degree centrality:\t {min(betweenness, key=betweenness.get)}')
print('')
closeness = dict(nx.closeness_centrality(S_tweets_del))
print(f'Node with the highest closeness degree centrality:\t {max(closeness, key=closeness.get)}')
print(f'Node with the lowest closeness degree centrality:\t {min(closeness, key=closeness.get)}')
print('')
eigenvector = dict(nx.eigenvector_centrality(S_tweets_del))
print(f'Node with the highest eigenvector degree centrality:\t {max(eigenvector, key=eigenvector.get)}')
print(f'Node with the lowest eigenvector degree centrality:\t {min(eigenvector, key=eigenvector.get)}')
print('')
load = dict(nx.load_centrality(S_tweets_del))
print(f'Node with the highest load degree centrality:\t\t {max(load, key=load.get)}')
print(f'Node with the lowest load degree centrality:\t\t {min(load, key=load.get)}')

-------------
Density:			 0.01052555168408827
Network diameter:		 25
Average shortest path length:	 10.110506194347657
Average degree:			 3.0208333333333335
-------------
Node with the highest degree centrality:		 90723
Node with the lowest degree centrality:			 87556

Node with the highest betweenese degree centrality:	 90723
Node with the lowest betweenese degree centrality:	 87556

Node with the highest closeness degree centrality:	 97273
Node with the lowest closeness degree centrality:	 79917

Node with the highest eigenvector degree centrality:	 85563
Node with the lowest eigenvector degree centrality:	 105171

Node with the highest load degree centrality:		 90723
Node with the lowest load degree centrality:		 87556
